In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.densenet import DenseNet201
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
import math
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
import os
import json
from collections import defaultdict
import keras
import tensorflow as tf

Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13604741915026706716
]


In [4]:

# params = {}
# params["train_dir"] = "train"
# params["test_dir"] = "test"
# params["batch_size"] = 8
# params["img_width"] = 221
# params["img_height"] = 221


# params["loss"] = "categorical_crossentropy"
# params["metrics"] = ['top_k_categorical_accuracy', 'accuracy']
# params["initial_epoch"] = 2
# params["final_epoch"] = 4
# params["workers"] = 8
# params["step_per_epoch"] = 32
# params["train_threshold"] = 0
# params["phase1_optimizer"] = "adam"

# params["model_list"] = ["InceptionV3", "xception", "InceptionResNetV2", "DenseNet121", "DenseNet169", "DenseNet201"]
# params["dropout_list"] = [0.1, 0.2, 0.3]
# params["dense_list"] = [512, 1024]

# with open("params.json", "w") as f:
#     json.dump(params, f)
    


In [5]:
params = {}

with open("params_manual.json", "r") as f:
    params = json.load(f)

params["train_size"] = sum([len(files) for r, d, files in os.walk( params["train_dir"] )])
params["test_size"] = sum([len(files) for r, d, files in os.walk( params["test_dir"] )])
params["classes"] = sum([len(d) for r, d, files in os.walk( params["train_dir"] )])
params["phase2_optimizer"] = SGD(lr=0.001, momentum=0.9)
params["dense_num"] = 2
params["dense2"] = {"num":params["classes"], "activation":"softmax"}    

In [6]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        params["train_dir"],
        target_size = (params["img_width"], params["img_height"]),
        batch_size = params["batch_size"],
        shuffle = True,
        class_mode = 'categorical')

val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = val_datagen.flow_from_directory(
        params["test_dir"],
        target_size = (params["img_width"], params["img_height"]),
        batch_size = params["batch_size"],
        shuffle = True,
        class_mode = 'categorical')

Found 8187 images belonging to 102 classes.
Found 957 images belonging to 102 classes.


In [7]:
def create_model():
    base_model=None
    if params["model"] == "InceptionV3":
        params["train_threshold"] = 249
        base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=None, input_shape=(params["img_width"], params["img_height"], 3))
    elif params["model"] == "xception":
        params["train_threshold"] = 106
        base_model = Xception(weights='imagenet', include_top=False, input_tensor=None, input_shape=(params["img_width"], params["img_height"], 3))
    elif params["model"] == "InceptionResNetV2":
        params["train_threshold"] = 727
        base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=None, input_shape=(params["img_width"], params["img_height"], 3))
    elif params["model"] == "DenseNet121":
        params["train_threshold"] = 403
        base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=None, input_shape=(params["img_width"], params["img_height"], 3))    
    elif params["model"] == "DenseNet169": 
        params["train_threshold"] = 571
        base_model = DenseNet169(weights='imagenet', include_top=False, input_tensor=None, input_shape=(params["img_width"], params["img_height"], 3))    
    elif params["model"] == "DenseNet201":
        params["train_threshold"] = 683
        base_model = DenseNet201(weights='imagenet', include_top=False, input_tensor=None, input_shape=(params["img_width"], params["img_height"], 3))    
    elif params["model"] == "ResNet50":
        params["train_threshold"] = 140
        base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=None, pooling=None, input_shape=(params["img_width"], params["img_height"], 3))    
    else:
        print("unknown model")

    count = 0
    modelx = base_model.output

    while count < params["dense_num"]:
        count += 1
        string = "dense"+str(count)

        if "pool" in params[string]:
            if params[string]["pool"] == "avg_poolx":
                modelx = GlobalAveragePooling2D(name=params[string]["pool"])(modelx)

        modelx = Dense(params[string]["num"], activation = params[string]["activation"])(modelx)
    
        if "dropout" in params[string]:
            modelx = Dropout(params[string]["dropout"])(modelx)
        
    model = Model(inputs=base_model.input, output=modelx)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(loss=params["loss"], optimizer=params["phase1_optimizer"], metrics=params["metrics"])
    
    return model

In [ ]:
class log_history(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        pass
     
    def on_batch_end(self, batch, logs={}):     
        params["batch"]["loss"].append(str(logs.get('loss')))        
        params["batch"]["top_k_categorical_accuracy"].append(str(logs.get('top_k_categorical_accuracy')))          
        params["batch"]["acc"].append(str(logs.get('acc')))
        #print(params["history"])
        with open(params["historypath"] + "/batch.json", "w") as f:
            json.dump(params["batch"], f)     
    
    def on_epoch_end(self, batch, logs={}):
        params["history"]["val_top_k_categorical_accuracy"].append(logs.get('val_top_k_categorical_accuracy'))
        params["history"]["val_loss"].append(logs.get('val_loss'))        
        params["history"]["loss"].append(logs.get('loss'))        
        params["history"]["top_k_categorical_accuracy"].append(logs.get('top_k_categorical_accuracy'))          
        params["history"]["acc"].append(logs.get('acc'))
        params["history"]["val_acc"].append(logs.get('val_acc')) 
        #print(params["history"])
        with open(params["historypath"] + "/epoch.json", "w") as f:
            json.dump(params["history"], f)         

In [ ]:
max_accuracy = 0
for model_name in params["model_list"]:
    params["model"] = model_name
    best_accuracy = 0

    for dropout_num in params["dropout_list"]:
        for dense_num in params["dense_list"]:
            sess = tf.Session()
            K.set_session(sess)
            params["dense1"] = {"num":dense_num, "dropout":dropout_num, "pool":"avg_poolx", "activation":"relu"}
            params["tensorpath"] = "./tensor/" + params['model'] + "/" + "dense_"+str(dense_num) + "_dropout_" + str(dropout_num)
            params["historypath"] = "./history/" + params['model'] + "/" + "dense_"+str(dense_num) + "_dropout_" + str(dropout_num)
            tbd = TensorBoard(log_dir=params["tensorpath"] ,  batch_size=params["batch_size"], write_graph=True )
            params["history"] = {}
            params["history"]["val_top_k_categorical_accuracy"] = []
            params["history"]["val_loss"] = []
            params["history"]["loss"] = []
            params["history"]["top_k_categorical_accuracy"] = []
            params["history"]["acc"] = []
            params["history"]["val_acc"] = []
            params["batch"] = {}
            params["batch"]["loss"] = []
            params["batch"]["acc"] = []
            params["batch"]["top_k_categorical_accuracy"] = []            
            model = create_model()
            epoch_history = log_history()
            if not os.path.exists(params['historypath']):
                os.makedirs(params['historypath'])
            
            history = model.fit_generator(generator=train_generator,
                         steps_per_epoch = params["step_per_epoch"],
                         epochs = params["initial_epoch"],
                         use_multiprocessing=False,
                         max_queue_size=10,
                         workers = params["workers"],     
                         validation_data = validation_generator,
                         callbacks=[tbd, epoch_history],
                         validation_steps = math.ceil(params["test_size"]  / params["batch_size"]))  
            
            for layer in model.layers[:params["train_threshold"]]:
               layer.trainable = False
            for layer in model.layers[params["train_threshold"]:]:
               layer.trainable = True
            params["phase2_optimizer"] = SGD(lr=0.001, momentum=0.9)                       
            model.compile(loss=params["loss"], optimizer=params["phase2_optimizer"], metrics=params["metrics"])
            history1 = model.fit_generator(generator=train_generator,
                         steps_per_epoch = params["step_per_epoch"] ,
                         epochs = params["final_epoch"] ,
                         initial_epoch= params["initial_epoch"] ,
                         use_multiprocessing=False,
                         max_queue_size=10,
                         workers = params["workers"],                                  
                         validation_data = validation_generator,
                         callbacks=[tbd, epoch_history],
                         validation_steps = math.ceil(params["test_size"]  / params["batch_size"]))
                   
            final_history = defaultdict(list)

            for d in (history.history, history1.history): # you can list as many input dicts as you want here
                for key, value in d.items():
                    for t in value:
                        final_history[key].append(t)
            with open(params["historypath"] + "/history.json", "w") as f:
                json.dump(final_history, f)  
                
            if (history1.history['val_acc'][-1] > best_accuracy):
                best_accuracy = history1.history['val_acc'][-1]
                model.save("model_" + params["model"] + ".h5")
                with open(params["model"] +"_history.json", "w") as f:
                    json.dump(final_history, f)

            if (history1.history['val_acc'][-1] > max_accuracy):
                max_accuracy = history1.history['val_acc'][-1]
                model.save("best_model" + ".h5")
                with open("best_history.json", "w") as f:
                    json.dump(final_history, f)
            del model
            sess.close()
            tf.reset_default_graph()

            K.clear_session()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/2
31/32 [============================>.] - ETA: 1s - loss: 4.7661 - top_k_categorical_accuracy: 0.2097